In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as graph
import copy

In [2]:
data = pd.read_csv('Mall_Customers_train.csv')

data

FileNotFoundError: [Errno 2] File b'Mall_Customers_train.csv' does not exist: b'Mall_Customers_train.csv'

In [ ]:
data_model_in = data[['Gender', 'Age', 'Annual Income (k$)']]

def cat_to_num(s):
    if s == 'Male':
        return 1
    elif s == 'Female':
        return 0

data_model_in['Gender_num'] = data_model_in['Gender'].apply(cat_to_num)
data_model_in

In [ ]:
Age_mean = np.mean(data_model_in['Age'].values)
Age_std = np.std(data_model_in['Age'].values)

AnnualIncome_mean = np.mean(data_model_in['Annual Income (k$)'].values)
AnnualIncome_std = np.std(data_model_in['Annual Income (k$)'].values)

Gender_mean = np.mean(data_model_in['Gender_num'].values)
Gender_std = np.std(data_model_in['Gender_num'].values)

Gender_mean

In [ ]:
data_model_in['Age_norm'] = (data_model_in['Age'] - Age_mean)/Age_std
data_model_in['AnnualIncome_norm'] = (data_model_in['Annual Income (k$)'] - AnnualIncome_mean)/AnnualIncome_std
data_model_in['Gender_norm'] = (data_model_in['Gender_num'] - Gender_mean)/Gender_std

data_model_in

In [ ]:
data_model_norm_in = data_model_in[['Gender_norm', 'Age_norm', 'AnnualIncome_norm']]

data_model_norm_in

In [ ]:
k_fold = 10
block_size = int(data_model_norm_in.shape[0]/k_fold)

block_dict = {}
for i in range(k_fold):
    if i < k_fold - 1:
        block_dict[i] = data_model_norm_in.loc[i*block_size: (i+1)*block_size - 1]
    else:
        block_dict[i] = data_model_norm_in.loc[i*block_size: ]

block_dict[7]

In [ ]:
CV_dict = {}
data_dict = {}
for i in range(k_fold):
    data_dict['validation'] = block_dict[i]
    
    not_i = np.array(range(k_fold))
    not_i = np.delete(not_i, i)
    frames = []
    for j in not_i:
        frames.append(block_dict[j])
    
    data_dict['train'] = pd.concat(frames)
    
    CV_dict[i] = copy.deepcopy(data_dict)

CV_dict[0]['validation']

In [ ]:
CV_dict[0]['train'].values - CV_dict[0]['validation'].loc[0].values

In [ ]:
np.sum((CV_dict[0]['train'].values - CV_dict[0]['validation'].loc[0].values)**2, axis = 1)

In [ ]:
df_temp = copy.deepcopy(CV_dict[0]['train'])
df_temp['distance'] = np.sum((CV_dict[0]['train'].values - CV_dict[0]['validation'].loc[0].values)**2, axis = 1)

df_temp.sort_values(by = 'distance')

In [ ]:
idx = df_temp.sort_values(by = 'distance').index.values
idx

In [ ]:
K = 5

K_idx = idx[0: K]

mean = 0
for i in K_idx:
    mean += data.loc[i]['Spending Score (1-100)']

mean /= K

mean

In [ ]:
nearest = pd.DataFrame(columns = data.columns)
for i in K_idx:
    nearest.loc[i] = data.loc[i]
    
nearest

In [ ]:
model_prediction = mean

actual_prediction = data.loc[0]['Spending Score (1-100)']

pred_error = actual_prediction - model_prediction

validation_cost = pred_error**2

validation_cost